In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install barbar

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

import torchvision
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
import random
import numpy as np
import os

from barbar import Bar
%matplotlib inline

In [ ]:
EPOCH = 200
C = 100
BATCH = 1024
PATH = '/content/drive/My Drive/visual_media/no_aug'

writer = SummaryWriter(log_dir=PATH)

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

SEED = 2020
seed_everything(SEED)

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR100(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR100(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH,
                                         shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
net = torch.hub.load('pytorch/vision:v0.6.0', 'resnet18', pretrained=False).cuda()

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.6.0


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.1)

In [ ]:
for epoch in range(EPOCH):
    
    ################
    ### Training ###
    ################
    running_loss = 0.0
    for i, data in enumerate(Bar(trainloader)):
        inputs, labels = data[0].cuda(), data[1].cuda()

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print('[%d] loss: %.3f' %
          (epoch + 1, running_loss / 50000 * i))
    writer.add_scalar("loss", running_loss / 50000 * i, epoch)
    running_loss = 0.0


    ################
    ### Testing  ###
    ################
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data[0].cuda(), data[1].cuda()
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: %d %%' % (
        100 * correct / total))
    writer.add_scalar("test_accuracy", 100 * correct / total, epoch)

    torch.save(net.state_dict(), PATH + '/model_latest.h5')
    scheduler.step()

50000/50000: [===============================>] - ETA 2.7s
[1] loss: 0.225
Accuracy of the network on the 10000 test images: 15 %
50000/50000: [===============================>] - ETA 0.5s
[2] loss: 0.161
Accuracy of the network on the 10000 test images: 20 %
50000/50000: [===============================>] - ETA 0.5s
[3] loss: 0.141
Accuracy of the network on the 10000 test images: 24 %
50000/50000: [===============================>] - ETA 0.5s
[4] loss: 0.125
Accuracy of the network on the 10000 test images: 26 %
50000/50000: [===============================>] - ETA 0.5s
[5] loss: 0.111
Accuracy of the network on the 10000 test images: 27 %
50000/50000: [===============================>] - ETA 0.6s
[6] loss: 0.096
Accuracy of the network on the 10000 test images: 28 %
50000/50000: [===============================>] - ETA 0.5s
[7] loss: 0.082
Accuracy of the network on the 10000 test images: 28 %
50000/50000: [===============================>] - ETA 0.5s
[8] loss: 0.067
Accuracy of the

In [ ]:
torch.save(net.state_dict(), PATH + '/model.h5')

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].cuda(), data[1].cuda()
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 29 %
